# Final Exam

Exam is open book, open note, and open Google. You are not allowed outside
help from another person, however. All work, including coding, must be yours alone. Turn in this coding portion by downloading your completed Colab notebook as a .ipynb file and submitting it via Learning suite. To get full credit, the completed notebook should be able to run top to bottom, producing the results asked for in the prompt below.

## The Question

An important question in criminal justice is whether detaining defendants while they await trial affects their ultimate probability of conviction. In this final exam you will use machine learning to estimate the causal effect of pre-trial detention on conviction. In the [shared Econ 484 Google Drive "datasets" folder](https://drive.google.com/drive/folders/1gAPP5E0zb4XRd8Ey9eyJ48IIlCCutU9c?usp=sharing) you will find a dataset called "miami_example.csv" and the associated codebook "miami_defs.txt" that gives some information about each of the variables. The outcome variable is the indicator for conviction. The treatment variable is an indicator pre-trial detention. Possible instrumental variables are dummy variables for each judge. You will have to create the dummy variables from the judge id variable. Additional covariates are age, gender, race, prior offender indicator, number of counts, and indicators for type of crimes in the current charge.

## The Task

Estimate the causal effect of pre-trial detention on conviction in two ways:

1) Via OLS regression where you use machine learning to control for the additional covariates.

2) Via instrumental variables regression using dummy variables for bail judge  as instruments for pre-trial detention where you use machine learning to address the problems that can arise with many instruments (there are 166 judges). 

## Hints and Requirements

*   Thoroughly document your code with comments explaining what each part of your code is doing

*   Be sure to "print" all of the relevant results after estimating/calculating them

*   Use best practices that we have learned this semester, including pre-processing variables as necessary and choosing tuning parameters.

*   Hint: the dataset contains missing values for some of the variables and some of the observations. You may assume that observations are missing at random, and therefore restricting your dataset to observations for which no variables are missing is appropriate.

In [ ]:
import pandas as pd
import numpy as np
from sklearn import linear_model
from sklearn.model_selection import KFold

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
%cd /content/drive/MyDrive/Econ\ 484/datasets
# Read in data
df = pd.read_csv('miami_example.csv')
print(df.head())
print("Shape (by observations and variables respectively): {}".format(str(df.shape)))

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive/MyDrive/Econ 484/datasets
   any_guilty  detained  judgeid  age  female  white  black  \
0           1         1        1   21       0    0.0    1.0   
1           0         1        1   26       0    0.0    1.0   
2           0         1        1   20       0    0.0    1.0   
3           1         1        1   50       0    0.0    1.0   
4           0         1        1   45       0    1.0    0.0   

   prior_offender_1year  n_counts  any_assault  ...  _Ibailmonth_5  \
0                     0         2            0  ...              0   
1                     0         2            0  ...              0   
2                     0         4            0  ...              0   
3                     0         3            0  ...              0   
4                     0         1            0  ...              0   

   _Ibailmonth_6  _Ibailmonth_

In [ ]:
df.columns

Index(['any_guilty', 'detained', 'judgeid', 'age', 'female', 'white', 'black',
       'prior_offender_1year', 'n_counts', 'any_assault', 'any_attmurder',
       'any_burglary', 'any_drug', 'any_dui', 'any_fraud', 'any_mischief',
       'any_murder', 'any_rape', 'any_robbery', 'any_theft', 'any_trespass',
       'any_weapon', '_Ibailyear_2007', '_Ibailyear_2008', '_Ibailyear_2009',
       '_Ibailyear_2010', '_Ibailyear_2011', '_Ibailyear_2012',
       '_Ibailyear_2013', '_Ibailyear_2014', '_Ibailmonth_2', '_Ibailmonth_3',
       '_Ibailmonth_4', '_Ibailmonth_5', '_Ibailmonth_6', '_Ibailmonth_7',
       '_Ibailmonth_8', '_Ibailmonth_9', '_Ibailmonth_10', '_Ibailmonth_11',
       '_Ibailmonth_12', '_Ibaildow_6', '_Ihighest_c_1'],
      dtype='object')

In [ ]:
df.describe()

any_guilty      detained      judgeid           age        female  \
count  93227.000000  93227.000000  93227.00000  93227.000000  93227.000000   
mean       0.583661      0.671436     79.54038     35.685660      0.156038   
std        0.492954      0.469693     48.13661     12.472581      0.362894   
min        0.000000      0.000000      1.00000     18.000000      0.000000   
25%        0.000000      0.000000     37.00000     25.000000      0.000000   
50%        1.000000      1.000000     79.00000     34.000000      0.000000   
75%        1.000000      1.000000    122.00000     46.000000      0.000000   
max        1.000000      1.000000    166.00000     70.000000      1.000000   

              white         black  prior_offender_1year      n_counts  \
count  93082.000000  93082.000000          93227.000000  93227.000000   
mean       0.475570      0.524022              0.339194      1.634001   
std        0.499405      0.499425              0.473438      1.222868   
min        0.000000      0.000000              0.000000      1.000000   
25%        0.000000      0.000000              0.000000      1.000000   
50%        0.000000      1.000000              0.000000      1.000000   
75%        1.000000      1.000000              1.000000      2.000000   
max        1.000000      1.000000              1.000000     10.000000   

        any_assault  ...  _Ibailmonth_5  _Ibailmonth_6  _Ibailmonth_7  \
count  93227.000000  ...   93227.000000   93227.000000   93227.000000   
mean       0.184721  ...       0.087410       0.082455       0.086155   
std        0.388073  ...       0.282437       0.275058       0.280595   
min        0.000000  ...       0.000000       0.000000       0.000000   
25%        0.000000  ...       0.000000       0.000000       0.000000   
50%        0.000000  ...       0.000000       0.000000       0.000000   
75%        0.000000  ...       0.000000       0.000000       0.000000   
max        1.000000  ...       1.000000       1.000000       1.000000   

       _Ibailmonth_8  _Ibailmonth_9  _Ibailmonth_10  _Ibailmonth_11  \
count   93227.000000   93227.000000    93227.000000    93227.000000   
mean        0.086895       0.087099        0.080481        0.079902   
std         0.281683       0.281982        0.272038        0.271142   
min         0.000000       0.000000        0.000000        0.000000   
25%         0.000000       0.000000        0.000000        0.000000   
50%         0.000000       0.000000        0.000000        0.000000   
75%         0.000000       0.000000        0.000000        0.000000   
max         1.000000       1.000000        1.000000        1.000000   

       _Ibailmonth_12   _Ibaildow_6  _Ihighest_c_1  
count    93227.000000  93227.000000   93227.000000  
mean         0.078110      0.471741       0.525352  
std          0.268347      0.499203       0.499360  
min          0.000000      0.000000       0.000000  
25%          0.000000      0.000000       0.000000  
50%          0.000000      0.000000       1.000000  
75%          0.000000      1.000000       1.000000  
max          1.000000      1.000000       1.000000  

[8 rows x 43 columns]

In [ ]:
df = df.dropna()

In [ ]:
df.isnull().values.any() # Make sure there's no null values left after above ^ cleaning

False

In [ ]:
print(df.head())
print("Shape (by observations and variables respectively): {}".format(str(df.shape)))

   any_guilty  detained  judgeid  age  female  white  black  \
0           1         1        1   21       0    0.0    1.0   
1           0         1        1   26       0    0.0    1.0   
2           0         1        1   20       0    0.0    1.0   
3           1         1        1   50       0    0.0    1.0   
4           0         1        1   45       0    1.0    0.0   

   prior_offender_1year  n_counts  any_assault  ...  _Ibailmonth_5  \
0                     0         2            0  ...              0   
1                     0         2            0  ...              0   
2                     0         4            0  ...              0   
3                     0         3            0  ...              0   
4                     0         1            0  ...              0   

   _Ibailmonth_6  _Ibailmonth_7  _Ibailmonth_8  _Ibailmonth_9  _Ibailmonth_10  \
0              0              1              0              0               0   
1              0              0       

In [ ]:
y=df.loc[:,'any_guilty'] # The outcome variable is the indicator for conviction. 

d=df.loc[:,['detained']] #The treatment variable is an indicator pre-trial detention. 

X=df.drop(['any_guilty','detained', 'judgeid'],axis=1) # Additional covariates excluding judges

In [ ]:
lm=linear_model.LinearRegression() # Using linear regression as a reference point
lm.fit(d,y)
print("Simple regression effect of pre-trial detention on conviction: {:.3f}".format(lm.coef_[0]))

Simple regression effect of pre-trial detention on conviction: 0.259


In [ ]:
from sklearn.model_selection import GridSearchCV 
# define grid for alpha
alpha_grid = {'alpha': [.0000001, .000001, .00001, .0001],'max_iter': [1000]} # Any alpha above .0001 set all coefs to 0
grid_search = GridSearchCV(linear_model.Lasso(),alpha_grid,cv=5,return_train_score=True)
best_model=grid_search.fit(X,y)
print("Best alpha for Lasso: ",best_model.best_estimator_.get_params()['alpha'])

/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_coordinate_descent.py:648: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 6.526e+03, tolerance: 1.813e+00
  coef_, l1_reg, l2_reg, X, y, max_iter, tol, rng, random, positive
/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_coordinate_descent.py:648: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 7.015e+03, tolerance: 1.805e+00
  coef_, l1_reg, l2_reg, X, y, max_iter, tol, rng, random, positive
/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_coordinate_descent.py:648: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisa

Best alpha for Lasso:  1e-05


/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_coordinate_descent.py:648: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 3.975e+02, tolerance: 2.262e+00
  coef_, l1_reg, l2_reg, X, y, max_iter, tol, rng, random, positive


In [ ]:
lassoy = linear_model.Lasso(alpha=.00001, max_iter=1000,normalize=True).fit(X, y) # Lasso step 1 for y
lassoy.coef_!=0

/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_base.py:145: FutureWarning: 'normalize' was deprecated in version 1.0 and will be removed in 1.2.
If you wish to scale the data, use Pipeline with a StandardScaler in a preprocessing stage. To reproduce the previous behavior:

from sklearn.pipeline import make_pipeline

model = make_pipeline(StandardScaler(with_mean=False), Lasso())

If you wish to pass a sample_weight parameter, you need to pass it as a fit parameter to each step of the pipeline as follows:

kwargs = {s[0] + '__sample_weight': sample_weight for s in model.steps}
model.fit(X, y, **kwargs)

Set parameter alpha to: original_alpha * np.sqrt(n_samples). 
  FutureWarning,


array([ True,  True, False,  True,  True,  True,  True,  True,  True,
        True, False,  True, False, False,  True,  True,  True,  True,
       False,  True, False,  True,  True,  True,  True,  True,  True,
        True, False, False, False, False, False, False, False, False,
       False,  True, False,  True])

In [ ]:
alpha_grid = {'alpha': [.000001, .00001, .0001, .001, .01, .1],'max_iter': [1000]} 
grid_search = GridSearchCV(linear_model.Lasso(),alpha_grid,cv=5,return_train_score=True)
best_model=grid_search.fit(X,y)
print("Best alpha for Lasso: ",best_model.best_estimator_.get_params()['alpha'])

/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_coordinate_descent.py:648: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 2.732e+03, tolerance: 1.813e+00
  coef_, l1_reg, l2_reg, X, y, max_iter, tol, rng, random, positive
/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_coordinate_descent.py:648: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 6.160e+03, tolerance: 1.805e+00
  coef_, l1_reg, l2_reg, X, y, max_iter, tol, rng, random, positive
/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_coordinate_descent.py:648: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisa

Best alpha for Lasso:  1e-05


/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_coordinate_descent.py:648: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 3.975e+02, tolerance: 2.262e+00
  coef_, l1_reg, l2_reg, X, y, max_iter, tol, rng, random, positive


In [ ]:
lassod = linear_model.Lasso(alpha=best_model.best_estimator_.get_params()['alpha'], max_iter=1000,normalize=True).fit(X, d) # Lasso step 2 for d
lassoy.coef_!=0

/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_base.py:145: FutureWarning: 'normalize' was deprecated in version 1.0 and will be removed in 1.2.
If you wish to scale the data, use Pipeline with a StandardScaler in a preprocessing stage. To reproduce the previous behavior:

from sklearn.pipeline import make_pipeline

model = make_pipeline(StandardScaler(with_mean=False), Lasso())

If you wish to pass a sample_weight parameter, you need to pass it as a fit parameter to each step of the pipeline as follows:

kwargs = {s[0] + '__sample_weight': sample_weight for s in model.steps}
model.fit(X, y, **kwargs)

Set parameter alpha to: original_alpha * np.sqrt(n_samples). 
  FutureWarning,


array([ True,  True, False,  True,  True,  True,  True,  True,  True,
        True, False,  True, False, False,  True,  True,  True,  True,
       False,  True, False,  True,  True,  True,  True,  True,  True,
        True, False, False, False, False, False, False, False, False,
       False,  True, False,  True])

In [ ]:
Xunion=X.iloc[:,(lassod.coef_!=0) + (lassoy.coef_!=0)] # union of Xs to keep
Xunion.head() # view which remain

age  female  black  prior_offender_1year  n_counts  any_assault  \
0   21       0    1.0                     0         2            0   
1   26       0    1.0                     0         2            0   
2   20       0    1.0                     0         4            0   
3   50       0    1.0                     0         3            0   
4   45       0    0.0                     0         1            0   

   any_attmurder  any_burglary  any_drug  any_fraud  ...  _Ibailyear_2012  \
0              0             0         1          0  ...                0   
1              0             0         0          0  ...                0   
2              0             0         1          0  ...                0   
3              0             0         1          0  ...                0   
4              0             0         1          0  ...                0   

   _Ibailyear_2013  _Ibailyear_2014  _Ibailmonth_2  _Ibailmonth_4  \
0                0                0              0              0   
1                0                0              0              0   
2                0                0              0              0   
3                0                0              0              0   
4                0                0              0              0   

   _Ibailmonth_5  _Ibailmonth_11  _Ibailmonth_12  _Ibaildow_6  _Ihighest_c_1  
0              0               0               0            0              1  
1              0               0               1            1              1  
2              0               0               0            1              1  
3              0               0               0            0              1  
4              0               0               0            0              1  

[5 rows x 31 columns]

In [ ]:
dropped_Xs = X.columns.difference(Xunion.columns)
dropped_Xs # View lassoed out Xs

Index(['_Ibailmonth_10', '_Ibailmonth_3', '_Ibailmonth_6', '_Ibailmonth_7',
       '_Ibailmonth_8', '_Ibailmonth_9', 'any_dui', 'any_weapon', 'white'],
      dtype='object')

In [ ]:
rhs=pd.concat([d,Xunion],axis=1) # Combine all regressors 
fullreg=linear_model.LinearRegression().fit(rhs,y) 
print("PDS regression effect of pre-trial detention on conviction: {:.3f}".format(fullreg.coef_[0])) # Including covariates lessens effect

PDS regression effect of pre-trial detention on conviction: 0.157


In [ ]:
df

any_guilty  detained  judgeid  age  female  white  black  \
0               1         1        1   21       0    0.0    1.0   
1               0         1        1   26       0    0.0    1.0   
2               0         1        1   20       0    0.0    1.0   
3               1         1        1   50       0    0.0    1.0   
4               0         1        1   45       0    1.0    0.0   
...           ...       ...      ...  ...     ...    ...    ...   
93222           0         1      166   20       0    1.0    0.0   
93223           1         1      166   22       0    0.0    1.0   
93224           1         1      166   43       0    1.0    0.0   
93225           0         1      166   24       0    0.0    1.0   
93226           0         1      166   19       0    0.0    1.0   

       prior_offender_1year  n_counts  any_assault  ...  _Ibailmonth_5  \
0                         0         2            0  ...              0   
1                         0         2            0  ...              0   
2                         0         4            0  ...              0   
3                         0         3            0  ...              0   
4                         0         1            0  ...              0   
...                     ...       ...          ...  ...            ...   
93222                     1         1            0  ...              0   
93223                     0         1            0  ...              0   
93224                     0         1            0  ...              0   
93225                     0        10            0  ...              0   
93226                     1         1            0  ...              0   

       _Ibailmonth_6  _Ibailmonth_7  _Ibailmonth_8  _Ibailmonth_9  \
0                  0              1              0              0   
1                  0              0              0              0   
2                  0              0              0              0   
3                  1              0              0              0   
4                  0              1              0              0   
...              ...            ...            ...            ...   
93222              0              0              0              0   
93223              0              0              0              0   
93224              0              0              0              0   
93225              0              0              0              0   
93226              0              0              0              0   

       _Ibailmonth_10  _Ibailmonth_11  _Ibailmonth_12  _Ibaildow_6  \
0                   0               0               0            0   
1                   0               0               1            1   
2                   0               0               0            1   
3                   0               0               0            0   
4                   0               0               0            0   
...               ...             ...             ...          ...   
93222               1               0               0            0   
93223               1               0               0            0   
93224               1               0               0            0   
93225               1               0               0            0   
93226               0               1               0            0   

       _Ihighest_c_1  
0                  1  
1                  1  
2                  1  
3                  1  
4                  1  
...              ...  
93222              0  
93223              0  
93224              0  
93225              1  
93226              0  

[93082 rows x 43 columns]

### IV Regression

In [ ]:
df.columns

Index(['any_guilty', 'detained', 'judgeid', 'age', 'female', 'white', 'black',
       'prior_offender_1year', 'n_counts', 'any_assault', 'any_attmurder',
       'any_burglary', 'any_drug', 'any_dui', 'any_fraud', 'any_mischief',
       'any_murder', 'any_rape', 'any_robbery', 'any_theft', 'any_trespass',
       'any_weapon', '_Ibailyear_2007', '_Ibailyear_2008', '_Ibailyear_2009',
       '_Ibailyear_2010', '_Ibailyear_2011', '_Ibailyear_2012',
       '_Ibailyear_2013', '_Ibailyear_2014', '_Ibailmonth_2', '_Ibailmonth_3',
       '_Ibailmonth_4', '_Ibailmonth_5', '_Ibailmonth_6', '_Ibailmonth_7',
       '_Ibailmonth_8', '_Ibailmonth_9', '_Ibailmonth_10', '_Ibailmonth_11',
       '_Ibailmonth_12', '_Ibaildow_6', '_Ihighest_c_1'],
      dtype='object')

In [ ]:
df = pd.get_dummies(df, prefix='judge', columns=['judgeid']) # You will have to create the dummy variables from the judge id variable.
df

any_guilty  detained  age  female  white  black  prior_offender_1year  \
0               1         1   21       0    0.0    1.0                     0   
1               0         1   26       0    0.0    1.0                     0   
2               0         1   20       0    0.0    1.0                     0   
3               1         1   50       0    0.0    1.0                     0   
4               0         1   45       0    1.0    0.0                     0   
...           ...       ...  ...     ...    ...    ...                   ...   
93222           0         1   20       0    1.0    0.0                     1   
93223           1         1   22       0    0.0    1.0                     0   
93224           1         1   43       0    1.0    0.0                     0   
93225           0         1   24       0    0.0    1.0                     0   
93226           0         1   19       0    0.0    1.0                     1   

       n_counts  any_assault  any_attmurder  ...  judge_157  judge_158  \
0             2            0              0  ...          0          0   
1             2            0              0  ...          0          0   
2             4            0              0  ...          0          0   
3             3            0              0  ...          0          0   
4             1            0              0  ...          0          0   
...         ...          ...            ...  ...        ...        ...   
93222         1            0              0  ...          0          0   
93223         1            0              0  ...          0          0   
93224         1            0              0  ...          0          0   
93225        10            0              0  ...          0          0   
93226         1            0              0  ...          0          0   

       judge_159  judge_160  judge_161  judge_162  judge_163  judge_164  \
0              0          0          0          0          0          0   
1              0          0          0          0          0          0   
2              0          0          0          0          0          0   
3              0          0          0          0          0          0   
4              0          0          0          0          0          0   
...          ...        ...        ...        ...        ...        ...   
93222          0          0          0          0          0          0   
93223          0          0          0          0          0          0   
93224          0          0          0          0          0          0   
93225          0          0          0          0          0          0   
93226          0          0          0          0          0          0   

       judge_165  judge_166  
0              0          0  
1              0          0  
2              0          0  
3              0          0  
4              0          0  
...          ...        ...  
93222          0          1  
93223          0          1  
93224          0          1  
93225          0          1  
93226          0          1  

[93082 rows x 208 columns]

In [ ]:
Z = df.filter(like='judge',axis=1)
print('our instrument matrix is:\n',Z.head)

our instrument matrix is:
 <bound method NDFrame.head of        judge_1  judge_2  judge_3  judge_4  judge_5  judge_6  judge_7  judge_8  \
0            1        0        0        0        0        0        0        0   
1            1        0        0        0        0        0        0        0   
2            1        0        0        0        0        0        0        0   
3            1        0        0        0        0        0        0        0   
4            1        0        0        0        0        0        0        0   
...        ...      ...      ...      ...      ...      ...      ...      ...   
93222        0        0        0        0        0        0        0        0   
93223        0        0        0        0        0        0        0        0   
93224        0        0        0        0        0        0        0        0   
93225        0        0        0        0        0        0        0        0   
93226        0        0        0        0        0  

In [ ]:
ols = linear_model.LinearRegression() 
# Make new linear_model object, fit it on first stage 
ols_zero = ols.fit(Z,d)
dhat = ols_zero.predict(Z)
tsls = ols.fit(dhat,y)
print('2SLS coefficient: ', tsls.coef_)

2SLS coefficient:  [1.13003832]


In [ ]:
from sklearn.ensemble import RandomForestRegressor
rf = RandomForestRegressor(max_depth=2,max_features='sqrt')
# first grow random forest:
rf_fs=rf.fit(Z,np.ravel(d))
# now get random forest predictions to use as instrument:
iv_rf=np.reshape(rf_fs.predict(Z),(-1,1))
# do "first stage" using random forest predictions as instrument:
fs_rf=ols.fit(iv_rf,d)
dhat_rf=fs_rf.predict(iv_rf)
# finally, 2nd stage regression:
tsls_rf=ols.fit(np.reshape(dhat_rf,(-1,1)),y)
print('2SLS+Random Forest coefficient: ',tsls_rf.coef_)

2SLS+Random Forest coefficient:  [1.05802111]
